In [38]:
## Pasos:

#1. RandomForest - MDI
#2. Seleccionar mejores caractéristicas.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
from sklearn.base import clone
plt.style.use('ggplot')
pd.set_option("display.max_columns", 50)

%matplotlib inline

### 1. Leer datos pre-procesados

In [40]:
#Lectura del dataset

X_TRAIN_DF = pd.read_csv('data/02_data_intermedia/X_MI_entrenamiento_preprocesado.csv', header=0, sep=",")
X_TEST_DF  = pd.read_csv('data/02_data_intermedia/X_MI_pruebas_preprocesado.csv', header=0, sep=",")
Y_TRAIN_DF = pd.read_csv('data/02_data_intermedia/y_MI_entrenamiento_preprocesado.csv', header=0, sep=",")
Y_TEST_DF  = pd.read_csv('data/02_data_intermedia/y_MI_pruebas_preprocesado.csv', header=0, sep=",")

In [41]:
var_objetivo_lst = [
    'FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD',
    'A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN',
    'REC_IM','P_IM_STEN'
]

### 2. Selección de mejores características usando MDI y RandomForest

In [44]:
X_TRAIN_CARACTERIZADO_DF = pd.DataFrame()
X_TEST_CARACTERIZADO_DF  = pd.DataFrame()
for var_objetivo in var_objetivo_lst:
    
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"] == var_objetivo]
    X_test  = X_TEST_DF.loc[X_TEST_DF["var_objetivo"] == var_objetivo] 
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]== var_objetivo ,[var_objetivo]]
    
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    rf = RandomForestRegressor(80, n_jobs=-1, oob_score=True)
    rf.fit(X_train, y_train)

    print(rf.feature_importances_)

    importances = pd.DataFrame(
        {"columns":X_train.columns, 
         "importances":rf.feature_importances_}).sort_values("importances", ascending=False)

    print(importances.style.bar())

    columnas_importantes = importances.loc[importances.importances>=0.0009,"columns"].unique()
    X_train_caracterizado = X_train[columnas_importantes]
    X_test_caracterizado = X_test[columnas_importantes]
    X_train_caracterizado["var_objetivo"] = var_objetivo
    X_test_caracterizado["var_objetivo"]  = var_objetivo
    
    X_TRAIN_CARACTERIZADO_DF = pd.concat([X_TRAIN_CARACTERIZADO_DF,X_train_caracterizado], axis=0)
    X_TEST_CARACTERIZADO_DF  = pd.concat([X_TEST_CARACTERIZADO_DF,X_test_caracterizado], axis=0)


    print(f"El proceso de selección de características para la variable objetivo {var_objetivo} obtiene \
            un total de {len(columnas_importantes)} características.")

D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-44-d3de15a69c67>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


[2.54801778e-02 5.05129158e-02 2.04359637e-02 9.07569449e-03
 8.76881579e-03 3.67840784e-02 2.39663320e-02 5.43942699e-04
 1.04627667e-02 5.46936571e-03 4.97476436e-03 2.14273344e-03
 7.00088114e-05 1.60860257e-02 1.24102720e-03 1.97407893e-03
 7.92303338e-05 2.05475528e-05 0.00000000e+00 7.10893821e-04
 0.00000000e+00 4.03931867e-05 2.02241131e-05 2.07282461e-05
 1.00695795e-02 2.53838322e-03 2.60548759e-04 2.17285317e-03
 7.13678080e-03 5.38929803e-03 6.83906822e-04 3.02151754e-03
 1.63858230e-02 1.66558798e-02 5.73871428e-03 3.34978317e-04
 3.31907744e-03 2.65924120e-03 3.41349587e-05 8.32253929e-04
 1.31235317e-02 3.31807074e-02 1.12843609e-02 5.86059726e-03
 6.25685965e-03 1.08588484e-02 2.08620095e-03 5.41768572e-04
 1.18127303e-03 3.56279769e-03 1.72327651e-02 1.68478734e-02
 1.33632055e-02 3.96640693e-03 1.65421287e-03 1.19444159e-01
 1.15182254e-03 1.41702955e-03 1.07029033e-02 1.88148209e-03
 4.91426918e-04 2.43672314e-03 3.32144289e-05 7.56296297e-05
 1.00900222e-02 3.375831

<ipython-input-44-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-44-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo
D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

[1.37109963e-03 3.12436498e-03 5.26933213e-03 1.12141594e-03
 9.96880121e-04 4.28647215e-03 7.90907054e-02 3.76242688e-04
 9.68895103e-04 1.19908601e-02 1.22212469e-04 1.87044652e-05
 0.00000000e+00 2.84017153e-04 5.36908265e-05 0.00000000e+00
 0.00000000e+00 0.00000000e+00 9.22631555e-05 0.00000000e+00
 2.54374869e-05 5.16894376e-04 0.00000000e+00 9.54450228e-06
 1.50987407e-02 1.40519880e-02 1.89729896e-05 1.53341111e-03
 1.08034352e-02 1.98791485e-04 3.76597991e-05 5.62562068e-05
 4.62458221e-03 4.53792702e-02 4.40298827e-03 2.92423145e-04
 1.79002408e-04 9.24476918e-02 0.00000000e+00 1.73014643e-04
 2.85395420e-02 2.60609497e-03 3.07710533e-03 8.95089556e-03
 1.18627732e-03 2.27100486e-02 3.52161698e-03 1.99282138e-04
 6.72521554e-04 1.08279003e-03 4.44776208e-04 9.28571163e-03
 3.19545924e-03 4.25517372e-03 7.27484292e-03 1.25313262e-03
 3.84466254e-03 7.59862936e-03 6.96261136e-03 2.41909498e-03
 2.38257176e-02 5.22561982e-03 1.34190371e-02 8.08081695e-03
 3.21478604e-03 3.283925

<ipython-input-44-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-44-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo
D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

...

<ipython-input-44-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-44-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo
D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

[4.72242448e-03 4.40351425e-02 2.81338998e-02 5.14387658e-02
 1.12191430e-01 4.11490483e-02 9.98134513e-03 3.07691907e-03
 1.06774759e-02 6.41732674e-03 4.30418621e-03 1.17230047e-03
 1.84634948e-03 2.63574631e-03 1.05482613e-03 9.51025755e-05
 6.60272708e-04 1.70886358e-03 5.84432798e-04 2.13618323e-03
 1.59749835e-04 3.91042214e-04 3.99254498e-04 1.48926943e-03
 3.64506378e-03 5.35429077e-03 2.27407346e-03 4.03438059e-03
 1.09878023e-02 2.61574357e-03 1.86283851e-04 2.24151050e-03
 1.71411164e-02 3.12840757e-02 6.12079376e-03 3.21208002e-03
 2.95810090e-03 4.94750805e-04 1.86655164e-03 1.99684538e-03
 1.24515782e-02 2.00380016e-02 1.82587033e-02 7.90074387e-03
 2.24298381e-03 3.04751157e-03 2.65528170e-03 6.19487524e-04
 1.02762682e-03 4.41307411e-03 1.65884564e-03 5.49641131e-03
 2.12177232e-03 5.70468153e-03 4.80427564e-03 1.16280278e-03
 3.53372979e-03 1.24808903e-03 6.42406180e-04 6.01369828e-03
 1.12137111e-03 3.44559996e-03 3.41174835e-03 1.26205127e-03
 9.06953011e-04 2.549188

<ipython-input-44-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-44-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo
D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

[5.16617013e-03 2.75722678e-02 1.56513136e-02 3.14738270e-02
 6.62226020e-02 3.84140724e-02 7.11677496e-03 1.59192019e-03
 1.51497090e-02 7.97886663e-03 3.01403817e-03 5.98956291e-05
 1.60664232e-04 1.68872062e-03 5.19205646e-04 2.05314199e-05
 5.77430024e-05 2.06370959e-05 1.72590023e-04 2.26022339e-03
 2.01123062e-05 4.54621108e-05 1.98988594e-05 7.81111266e-04
 5.31389855e-03 9.04298702e-04 1.13993421e-03 3.93189834e-03
 1.57635663e-02 3.45153789e-04 0.00000000e+00 2.28467087e-03
 3.24959893e-02 2.08606893e-02 4.83341499e-03 3.81062447e-04
 1.64761751e-03 1.58549533e-04 2.47577524e-04 5.77244552e-04
 1.51067358e-01 5.34805484e-02 4.66564720e-02 9.52879196e-03
 4.09738226e-03 7.29165911e-03 1.27132801e-03 8.08139972e-03
 5.04758867e-03 8.27033060e-03 4.54285967e-03 6.08307771e-03
 1.49376366e-03 7.16657949e-03 3.98958772e-03 1.43383559e-03
 5.28641320e-04 1.17024675e-03 1.09781403e-03 6.70714695e-04
 6.05901206e-04 1.14090903e-03 2.32590816e-03 6.63224980e-04
 1.05788211e-03 5.532434

<ipython-input-44-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-44-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo


### <u>Nota:</u> 
<mark>En este experimento se considera valores mayores a 0.0009 porque ratios de importancia menores no aportan o 
aportan muy poco al modelo. Adicionalmente, se consideró este ratio de corte con el fin de no perder
información significativa para el modelo. </mark>

### 3. Guardar resultados de la selección de características

In [45]:
X_TRAIN_CARACTERIZADO_DF.to_csv('data/03_entrada_modelo/X_MI_entrenamiento_caracterizado.csv',index=False)
X_TEST_CARACTERIZADO_DF.to_csv('data/03_entrada_modelo/X_MI_pruebas_caracterizado.csv',index=False)

Y_TRAIN_DF.to_csv('data/03_entrada_modelo/y_MI_entrenamiento_caracterizado.csv',index=False)
Y_TEST_DF.to_csv('data/03_entrada_modelo/y_MI_pruebas_caracterizado.csv',index=False)